# CNN Transfer&Train&Predict

# Init Config Setup

In [2]:
import requests
import json, os

url = "{0}:{1}".format(os.environ['HOSTNAME'] , "8000")

nn_id = "mro001"
biz_cate = "ERP"
biz_sub_cate = "MRO"
nn_title = "MRO Image Classification"
nn_desc = "MRO Image Classification"
nn_wf_ver_info = "MRO Image Classification"
use_flag = "Y"
dirstr = "purpose?"
config = "N"
network_type = "cnn"
node_sub_menu = "data_image"

# get Active workflow version
wf_ver_id = 0
if wf_ver_id == 0:
    resp = requests.get('http://' + url + '/api/v1/type/common/target/nninfo/nnid/'+nn_id+'/version/')
    data = json.loads(resp.json())

    # get Max workflow version
    for config in data:
        if config["fields"]["nn_wf_ver_id"] > wf_ver_id:
            wf_ver_id = config["fields"]["nn_wf_ver_id"]

wf_ver_id = str(wf_ver_id)

print("Network ID="+nn_id)
print("Workflow Version="+wf_ver_id)
print("")

def spaceprint(val, cnt):
    leng = len(str(val))
    cnt = cnt - leng
    restr = ""
    for i in range(cnt):
        restr += " "
    restr = restr+str(val)
    return restr

node = "netconf_node"
resp = requests.put('http://' + url + '/api/v1/type/wf/state/netconf/detail/cnn/nnid/'+nn_id+'/ver/'+wf_ver_id+'/node/'+node+'/',
                     json={
                         "param":{"epoch": 2
                                  ,"traincnt": 2
                                  ,"batch_size":1000000
                                  ,"predictcnt": 10
                         },
                         "config": {"num_classes":10,
                                    "learnrate": 0.001,
                                    "layeroutputs":32,
                                    "net_type":"cnn",
                                    "eval_type":"category",
                                    "optimizer":"AdamOptimizer" #RMSPropOptimizer
                                     }
                         ,"layer1": {"active": "relu",
                                     "cnnfilter": [3, 3],
                                     "cnnstride": [1, 1],
                                     "maxpoolmatrix": [2, 2],
                                     "maxpoolstride": [2, 2],
                                     "padding": "SAME",
                                     "droprate": "0.8",
                                     "layercnt":2
                                    }
                         ,"layer2": {"active": "relu",
                                     "cnnfilter": [3, 3],
                                     "cnnstride": [1, 1],
                                     "maxpoolmatrix": [1, 1],
                                     "maxpoolstride": [2, 2],
                                     "padding": "SAME",
                                     "droprate": "0.8",
                                     "layercnt":1
                                    }
                         ,"out": {"active": "softmax",
                                   "node_out": 625,
                                   "padding": "SAME"
                                }
                         ,"labels":[]
                        })
netconf = json.loads(resp.json())
print("Network Config : {0}".format(netconf))
print("")

node = "datasrc"
resp = requests.put('http://' + url + '/api/v1/type/wf/state/imgdata/src/local/form/file/prg/source/nnid/'+nn_id+'/ver/'+wf_ver_id+'/node/'+node+'/',
                     json={
                            "preprocess": {"x_size": 32,
                                           "y_size": 32,
                                           "channel":3,
                                           "filesize": 1000000,
                                           "yolo": "n"}
                     })
dataconf = json.loads(resp.json())
print("Train Data Config : {0}".format(dataconf))
print("")

node = "eval_node"
resp = requests.put('http://' + url + '/api/v1/type/wf/state/netconf/detail/cnn/nnid/'+nn_id+'/ver/'+wf_ver_id+'/node/'+node+'/',json={})
evalconf = json.loads(resp.json())
print("Eval Config : {0}".format(evalconf))
print("")

node = 'evaldata'
resp = requests.put('http://' + url + '/api/v1/type/wf/state/imgdata/src/local/form/file/prg/source/nnid/'+nn_id+'/ver/'+wf_ver_id+'/node/'+node+'/',json={
                            "preprocess": {"x_size": 32,
                                           "y_size": 32,
                                           "channel":3,
                                           "filesize": 1000000,
                                           "yolo": "n"}
                     })
edataconf = json.loads(resp.json())
print("Eval Data Config : {0}".format(edataconf))
print("")

Network ID=mro001
Workflow Version=4

Network Config : {'layer1': {'layercnt': 2, 'active': 'relu', 'maxpoolmatrix': [2, 2], 'maxpoolstride': [2, 2], 'droprate': '0.8', 'cnnstride': [1, 1], 'cnnfilter': [3, 3], 'padding': 'SAME'}, 'config': {'eval_type': 'category', 'layeroutputs': 32, 'learnrate': 0.001, 'num_classes': 10, 'net_type': 'cnn', 'optimizer': 'AdamOptimizer'}, 'layer2': {'layercnt': 1, 'active': 'relu', 'maxpoolmatrix': [1, 1], 'maxpoolstride': [2, 2], 'droprate': '0.8', 'cnnstride': [1, 1], 'cnnfilter': [3, 3], 'padding': 'SAME'}, 'param': {'predictcnt': 10, 'batch_size': 1000000, 'epoch': 2, 'traincnt': 2}, 'out': {'active': 'softmax', 'node_out': 625, 'padding': 'SAME'}, 'labels': ['Pump Accessories', 'Protection Relay', 'Pressure Transmitter', 'Flexible Hose', 'Aux Relay', 'Hydraulic Pump', 'Circuit Breaker', 'Oil Seal_Bearing Isolator', 'Manual Switch', 'Control Valve', 'Printed Circuit Board', 'Integrated Circuit', 'Hose', 'Laboratory Supply', 'Varied Seal', 'Roller 

# Trainning & Eval Data Transfer
(CNN Network Train을 할 수 있게 Data를 특정 장소에 전송해 준다.)<br>

아래 API의 경우 Test Data인지 Train Data인지 선택할수 있는 Node 정보를 모두 가져와 준다.

In [3]:
# get workflow node list info (train, eval)
resp = requests.get('http://' + url + '/api/v1/type/wf/state/data/detail/upload/file/nnid/'+nn_id+'/ver/'+wf_ver_id+'/dir/'+node_sub_menu+'/')
datalist = json.loads(resp.json())

print("get file node list")
for node in datalist:
    print(datalist[node])
print("")

get file node list
{'nn_wf_node_name': 'evaldata', 'nn_wf_node_desc': 'Evaluation Data Node', 'nn_wf_node_id': 'nn00011_2_evaldata'}
{'nn_wf_node_name': 'datasrc', 'nn_wf_node_desc': 'Train Data Node', 'nn_wf_node_id': 'nn00011_2_datasrc'}



아래 API는 한개의 파일을 Train Folder & Test Folder 에 넣어준다.

In [4]:
# Train Folder에 Data를 위치 시킬지 Eval Folder에 Data를 위치 시킬지 결정후 전송해야 한다.
print("create file node list")
for node in datalist:
    typepath = datalist[node]["nn_wf_node_name"]

    files = {'file1': open('/home/dev/hoyai/demo/data/cat_vs_dog.zip','rb'),
             'file2': open('/home/dev/hoyai/demo/data/sample_cnn_img.zip','rb')}

    resp = requests.post('http://' + url + '/api/v1/type/wf/state/data/detail/upload/file/nnid/'+nn_id+'/ver/'+wf_ver_id+'/dir/'+typepath+'/',
                         files=files)
    data = json.loads(resp.json())
    print(data)

create file node list
{'File0': {'File': 'File Create.(/hoya_src_root/nn00011/2/evaldata/sample_cnn_img.zip)'}, 'File1': {'File': 'File Create.(/hoya_src_root/nn00011/2/evaldata/sample_cnn_img.zip)'}}
{'File0': {'File': 'File Create.(/hoya_src_root/nn00011/2/datasrc/sample_cnn_img.zip)'}, 'File1': {'File': 'File Create.(/hoya_src_root/nn00011/2/datasrc/sample_cnn_img.zip)'}}


# CNN Network Training
(CNN Network Training을 실행한다.)<br> 

In [3]:
resp = requests.post('http://' + url + '/api/v1/type/runmanager/state/train/nnid/'+nn_id+'/ver/'+wf_ver_id+'/')
data = json.loads(resp.json())

for train in data:
    if train != None and train != "" and train != {} and train != "status" and train != "result":
        try:
            for tr in train["TrainResult"]:
                print(tr)
        except:
            maxcnt = 0
            line = ""
            for label in train["labels"]:
                if maxcnt<len(label):
                    maxcnt = len(label)

            for i in range(len(train["labels"])):
                for j in range(maxcnt+4):
                    line += "="

            label_sub = []
            for label in train["labels"]:
                label = spaceprint(label,maxcnt)
                label_sub.append(label)

            space = ""
            for s in range(maxcnt):
                space +=" "

            print(space, label_sub)
            print(space, line)
            for i in range(len(train["labels"])):
                truecnt = 0
                totcnt = 0
                predict_sub = []
                for j in range(len(train["predicts"][i])):
                    pred = spaceprint(train["predicts"][i][j],maxcnt)

                    predict_sub.append(pred)
                    totcnt += int(pred)
                    # print(train["labels"].index(train["labels"][i]))
                    if train["labels"].index(train["labels"][i]) == j:
                        truecnt = int(pred)
                if totcnt == 0:
                    percent = 0
                else:
                    percent = round(truecnt/totcnt*100,2)
                print(spaceprint(train["labels"][i],maxcnt), predict_sub, str(percent)+"%")

['Trainning ..................................................']
['Global Step: 453, Training Batch Accuracy: 11.37%, Cost: 3.65533']

['Trainning ..................................................']
['Global Step: 455, Training Batch Accuracy: 11.36%, Cost: 3.6428']

                             ['            Pump Accessories', '            Protection Relay', '        Pressure Transmitter', '               Flexible Hose', '                   Aux Relay', '              Hydraulic Pump', '             Circuit Breaker', '   Oil Seal_Bearing Isolator', '               Manual Switch', '               Control Valve', '       Printed Circuit Board', '          Integrated Circuit', '                        Hose', '           Laboratory Supply', '                 Varied Seal', '              Roller Bearing', '                        Fuse', '              Shut-Off Valve', '             Hydraulic Valve', '                   Flowmeter', '                  Controller', ' Varied Measuring Instrument

# CNN Network Predict
(CNN Network Predict를 실행한다.)<br>

In [4]:
files = {
         'files000001':  open('/home/dev/hoyai/demo/data/airplane/1air.jpg','rb')
        ,'files000002':  open('/home/dev/hoyai/demo/data/airplane/2air.jpg','rb')
        ,'files000003':  open('/home/dev/hoyai/demo/data/bolt/1bolt.jpg','rb')
        ,'files000004':  open('/home/dev/hoyai/demo/data/bolt/2bolt.jpg','rb')
        ,'files000005':  open('/home/dev/hoyai/demo/data/car/1car.jpg','rb')
        ,'files000006':  open('/home/dev/hoyai/demo/data/car/2car.jpg','rb')
        ,'files000007':  open('/home/dev/hoyai/demo/data/glove/1glove.jpg','rb')
        ,'files000008':  open('/home/dev/hoyai/demo/data/glove/2glove.jpg','rb')
        ,'files000009':  open('/home/dev/hoyai/demo/data/motor/1motor.jpg','rb')
        ,'files000010':  open('/home/dev/hoyai/demo/data/motor/2motor.jpg','rb')
        }
restURL = 'http://' + url + '/api/v1/type/service/state/predict/type/'+network_type+'/nnid/'+nn_id+'/ver/'+wf_ver_id+'/'

resp = requests.post(restURL,
                     files=files
                     )
data = json.loads(resp.json())

for train in data:
    print("FileName = "+train)
    print(data[train]['key'])
    print(data[train]['val'])
    print('')

FileName = 2car.jpg
['Power_Control Cable', 'Compressor Accessories', 'Controller', 'Aux Relay', 'Shut-Off Valve', 'Pressure Switch', 'AC Geared Motor', 'Manual Switch', 'Solenoid Valve', 'Protection Relay']
[4.202712, 4.147773, 4.103412, 3.9983709999999997, 3.650054, 3.6216749999999998, 3.1252670000000005, 3.103244, 3.090541, 2.878005]

FileName = 1glove.jpg
['Pressure Transmitter', 'Manual Switch', 'Encoder', 'AC Induction Motor', 'Centrifugal Pump', 'Pressure Switch', 'Varied Measuring Instrument', 'DC Power Supply', 'Varied Sensor', 'Varied Seal']
[1.688259, 1.687261, 1.681761, 1.6802540000000001, 1.679918, 1.679706, 1.679004, 1.67829, 1.678212, 1.6781069999999998]

FileName = 1bolt.jpg
['Pipe Coupling', 'Position Sensor', 'DC Power Supply', 'Elbow', 'Encoder', 'PIPE', 'Centrifugal Pump', 'Hydraulic Valve', 'Nozzle', 'Pneumatic Cylinder']
[13.714473, 9.969079, 7.280759000000001, 5.576111, 5.4021859999999995, 3.112856, 2.914908, 2.873561, 2.782186, 2.662649]

FileName = 1motor.jpg
[